In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option('display.max_colwidth', -1)

%matplotlib inline

/home/bird/Documents/Code/tbrain20_AML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [5]:
data_path = "data/tbrain_train_final_0610.csv"

df = pd.read_csv(data_path)
df.head()

,news_ID,hyperlink,content,name
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35facebook commentFONT SIZEICON PRINT78981DD6-B6FA-435C-B969-0D6ABD01805C0「時間序列與量化交易研討會」邀集專家學者進行量化交易、智能投資等相關議題研討。(圖:鄭心芸攝)相關個股元大期6023近年來投資市場波動越來越明顯 ### 省略內文 ### 都有非常巨大的助益。,[]
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北市政府今日下午也跨局處召開「外送人員保障專案會議」。圖／台北市勞動局提供,[]
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文｜李育材攝影｜李育材全文朗讀00:00 / 00:00君鴻飯店快速遭法拍 ### 省略內文 ### 亦即本次拍賣事件均屬合法。更新時間｜2019.10.10 09:14,[]
3,4,https://www.chinatimes.com/realtimenews/20190917000619-260106,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。 (中國時報),[]
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9HN03.jsp?crtid=PTD&filenm=ECE5DAECAF563DA171E34B03FC5185742FC90F879191D5DF,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家親108年度司家催字第58號附件：主旨：公告本院108年度司家催字第58號公示催告事件裁定。依據：家事事件法第137條、第138條、第139條準用第130條第3項至第5項、臺灣地區及大陸地區人民關係條例第68條第1項、第3項、退除役官兵死亡無人繼承遺產管理辦法第6條。公告事項：一、本案遺產管理人為國軍退除役官兵輔導委員會屏東榮譽國民之家（住址：屏東縣內埔鄉○○路000號）。二、本件係對被繼承人陳世恒（男、民國16年10月6日生、身分證統一編號：F10****885號、108年8月29日死亡、生前最後住所地：屏東縣內埔鄉○○路000號）之繼承人、債權人及受遺贈人為公示催告。三、被繼承人陳世恒之繼承人（除大陸地區人民應依臺灣地區與大陸地區人民關係條例之規定外）、債權人及受遺贈人 ### 省略內文 ### 僅得就賸餘遺產行使其權利。五、本件裁定正本黏貼於本院公告處。核章欄：書記官科長司法事務官書記官：沈君融股別：親,[]


In [15]:
u_name = df["name"].tolist()
u_name = {name.strip("]").strip("[").strip("'") for name in u_name}
print(len(df))
print(u_name)

5023
{'', "楊肇忠', '鄧福鈞", "吳秋華', '廖麗櫻", '朱傑麟', "江盈均', '江志堅', '許志遠', '蔡世誠", "李孟謙', '于曉艷', '于曉燕", "朱國榮', '劉慶珠", '翁茂鍾', "羅雅美', '邱彰信', '謝世謙', '吳宗憲', '張恒嘉", "曾昭榮', '鄒春香', '鄒官羽', '陳効亮', '姚柏丞', '梁柱', '馮一塵', '林夢珍', '蘇雅玲', '杜嘉珊', '劉人鳳", '吳松薰', "趙瑩潔', '郭展緁", "林維屹', '林孟良', '林孟函', '郭雨隴', '彭紹軒', '林劼", "劉威甫', '張桂銘", '柯賜海', 'Sreenack', "黃春發', '賴粵興', '鄭達騰', '林健標', '何歡", "施允澤', '施建新', '吳茂達", '陳美蘭', '顏政德', "陳仕修', '徐仲榮", "崔明禮', '汪惠南', '周榮華", '陳柏霖', '洪玉鳳', "陳偉', '黃載文", '賴文德', "趙宏翰', '陳文政", "鄭銘坤', '李岳怡", '游朝旭', '陳淳伍', '鮑廣廷', "呂紹甫', '蔣楚明", '張智凱', '謝宥宏', '許正雄', 'Denis Katana', '楊治渝', "林秉勳', '林文良', '盧文凱", '楊士弘', "連千毅', '鄭又仁", "林文楨', '吳俊宗", '鄭聖儒', '黃俊策', '吳宗憲', '徐豪雄', '李維凱', "林昱伯', '周漢祥', '林煒智', '林睿耆", "于求潭', '孫林莫", "陳澄玄', '張金素', '賈翔傑', '陳子俊", "陳耀東', '葉麗珍', '葉麗貞', '趙鈞震", "黃天一', '吳洛瑜', '黃啟禎", "林大福', '邱平舜', '黃建彰", '賴素如', "楊昇穎', '周宗賢", "趙藤雄', '趙信清', '李克毅", "郭明枝', '鄧文聰", '黃薪哲', "孔繁仁', '楊富巖', '戴盛世", '戴家秀', "彭振源', '王澤生', '楚瑞芳", "林煜傑', '李育英", "林進隆', '徐炳清", '洪勝明', '郭政權', "張平沼', '林瑞足', '張國安', '陳淑珠", "倪菲爾', '凌鳳琴', '李泗源', 